In [1]:
# Here we predict only stocks direction i.e. down or up and that too for one step only.That should make the problem easier

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC  # we are importing classifiers and not regressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_percentage_error,r2_score

In [8]:
!curl -O https://lazyprogrammer.me/course_files/sp500sub.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0 24.2M    0 30795    0     0  18019      0  0:23:33  0:00:01  0:23:32 18040
  2 24.2M    2  542k    0     0   198k      0  0:02:05  0:00:02  0:02:03  198k
  8 24.2M    8 2126k    0     0   585k      0  0:00:42  0:00:03  0:00:39  585k
 23 24.2M   23 5774k    0     0  1209k      0  0:00:20  0:00:04  0:00:16 1209k
 37 24.2M   37 9214k    0     0  1624k      0  0:00:15  0:00:05  0:00:10 2128k
 47 24.2M   47 11.6M    0     0  1703k      0  0:00:14  0:00:07  0:00:07 2247k
 63 24.2M   63 15.3M    0     0  2039k      0  0:00:12  0:00:07  0:00:05 3049k
 81 24.2M   81 19.7M    0     0  2337k      0  0:00:10  0:00:08  0:00:02 3605k
 95 24.2M   95 23.0M    0     0  2455k      0  0:00

In [9]:
df0=pd.read_csv('sp500sub.csv',index_col='Date',parse_dates=True)
df0.head()

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2010-01-04,9.22,9.51,9.17,9.50,9.50,1865400.0,INCY
2010-01-05,9.51,10.29,9.45,10.27,10.27,7608900.0,INCY
2010-01-06,10.38,11.09,10.35,11.00,11.00,8046700.0,INCY
2010-01-07,11.00,11.06,10.62,10.82,10.82,3680300.0,INCY
2010-01-08,10.82,11.00,10.75,10.94,10.94,1529300.0,INCY


In [10]:
df=df0[df0['Name']=='IBM'][['Close']].copy()

In [11]:
df['LogClose']=np.log(df['Close'])

In [12]:
df['LogReturn']=df['LogClose'].diff()

In [13]:
Ntest=252
train=df.iloc[:-Ntest]
test=df.iloc[-Ntest:]

In [14]:
# Make supervised dataset
# Let's see if we can use T past values to predict the next value

series=df['LogReturn'].to_numpy()[1:] # Numpy array are little easier to index
target=(series>0)*1
# It starts from 1 as due to taking the first difference the first value is zero
T=21 # We set the number of lags.
X=[] # We also create empty lists to store inputs and targets
Y=[]

for t in range(len(series)-T):
    x=series[t:t+T] # This is our input which is of the time series of size T
    X.append(x)
    y=target[t+T] # This is to compute the target which is the next value of the time series
    Y.append(y)
    
# We convert both the inputs and targets to the numpy array, which are easy to index    
X=np.array(X).reshape(-1,T)
Y=np.array(Y)
N=len(X)
print('X.shape',X.shape,'Y.shape',Y.shape)

X.shape (2241, 21) Y.shape (2241,)


In [15]:
# To split our input and targets into train and test
Xtrain,Ytrain=X[:-Ntest],Y[:-Ntest]
Xtest,Ytest=X[-Ntest:],Y[-Ntest:]

In [22]:
# To fit a linear regression
lr=LogisticRegression()
lr.fit(Xtrain,Ytrain)
lr.score(Xtrain,Ytrain)

0.5093011563599799

In [23]:
lr.score(Xtest,Ytest)

0.49603174603174605

In [24]:
svc=SVC()
svc.fit(Xtrain,Ytrain)
svc.score(Xtrain,Ytrain)

0.7551533433886375

In [25]:
svc.score(Xtest,Ytest)

0.49603174603174605

In [26]:
# This is the same as coin toss, so no big deal
rf=RandomForestClassifier()
rf.fit(Xtrain,Ytrain)
rf.score(Xtrain,Ytrain)

1.0

In [27]:
rf.score(Xtest,Ytest)

0.44841269841269843

In [28]:
# Does worse than coin test on test set